In [ ]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm, trange
import geopandas as gpd

plt.style.use("default")

In [ ]:
from preprocess import X_freq, X_sev, df, sev_mask, w_freq, w_sev, y_freq, y_sev, all_features

In [ ]:
nclaims_summary = df.groupby("nclaims").agg("size").to_frame("Count")
nclaims_summary["Freq"] = (nclaims_summary["Count"] / nclaims_summary["Count"].sum()).map("{:.2%}".format)
nclaims_summary

In [ ]:
from model import FFNNPremiumModel, GAMPremiumModel, GBMPremiumModel, GLMPremiumModel, MTMoENNPremiumModel, MTNNPremiumModel

models = {n: {fold: m.load(f"{m.__name__}_fold_{fold}.pkl") 
              for fold in range(1, 7)} for n, m in [
    ("GAM", GAMPremiumModel), 
    ("GLM", GLMPremiumModel), 
    ("GBM", GBMPremiumModel), 
    ("FFNN", FFNNPremiumModel), 
    ("MTNN", MTNNPremiumModel), 
    ("MTMoE", MTMoENNPremiumModel)
]}


In [ ]:
df["F"] = df["nclaims"] / df["expo"]
for agg_name, agg_col, agg_func in [
    ("Expected Frequency", "F", "mean"), 
]:
    for feature_type in ["category", "int"]:
        cols = np.intersect1d(df.select_dtypes(include=[feature_type]).columns, all_features)
        fig, axes = plt.subplots(nrows=1, ncols=len(cols), figsize=(15, 3), constrained_layout=True)
        for col, ax in zip(cols, axes):
            x, counts = np.unique(df[col], return_counts=True)
            y = df.groupby(col, observed=True).agg({agg_col: agg_func})[agg_col]
            ax.bar(x, y, alpha=0.7)
            ax.grid(axis='y', linestyle='--', alpha=0.4)
            ax.set_xlabel(col)
            ax.set_ylabel(agg_name)
            ax.set_title(f"{agg_name} per {col}")
        fig.savefig(f"{agg_name.lower().replace(' ', '_')}_{feature_type}.pdf", bbox_inches='tight')

In [ ]:
geodata = gpd.read_file("https://raw.githubusercontent.com/arneh61/Belgium-Map/master/Gemeenten.json", columns=["geometry", "NAME_4"]) \
    .set_crs(epsg=4326) \
    .rename(columns={"NAME_4": "name"}) \
    .merge(pd.read_csv("data/postcodes.csv", index_col='name'), on='name') \
    .merge(df[['expo', 'nclaims', 'amount', 'ageph', 'power', 'agec', 'bm', 'postcode']].groupby('postcode').agg('mean').reset_index().set_index('postcode'),
        on='postcode', how='left')

cols = geodata.columns[3:]
fig, axes = plt.subplots(2, 3, figsize=(16, 10), sharex=True, sharey=True, constrained_layout=True)

for ax, col in zip(axes.flatten(), cols):
    geodata.plot(ax=ax, column=col, legend=False, cmap="Blues")
    ax.set_title(f'Mean {col} per Region')
    vmin = geodata[col].min()
    vmax = geodata[col].max()
    sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
    cbar = plt.colorbar(sm, ax=ax)
    ax.set_axis_off()

fig.savefig("geodata_map.pdf", bbox_inches='tight')

In [ ]:
from pygam.terms import TermList
from pygam import GAM

def plot_partial_dependence(gam: GAM, X: pd.DataFrame, save_prefix=None):
    terms = gam.terms
    assert isinstance(terms, TermList)

    numerical_terms: list[int] = []
    tensor_terms: list[int] = []
    categorical_terms: list[int] = []
    for term_idx, term in enumerate(terms):
        if term.isintercept:
            continue
        match term.dtype:
            case 'categorical':
                categorical_terms.append(term_idx)
            case 'numerical':
                numerical_terms.append(term_idx)
            case ['numerical', 'numerical']:
                tensor_terms.append(term_idx)

    print("Categorical Terms:", categorical_terms)
    print("Numerical Terms:", numerical_terms)
    print("Tensor Terms:", tensor_terms)

    fig_numerical, fig_categorical, fig_tensor = None, None, None

    if len(numerical_terms) > 0:
        fig_numerical, axis = plt.subplots(figsize=(10, 2), nrows=1, ncols=len(numerical_terms))
        # plot numerical terms as 2D curve
        for i, term_idx in enumerate(numerical_terms):
            term = terms[term_idx]
            feature_name = X.columns[term.feature]
            feature_range = np.percentile(X[feature_name].dropna(), [2.5, 97.5])
            # create a grid of values for the feature
            XX = np.zeros((100, len(X.columns)))
            XX[:, term.feature] = np.linspace(*feature_range, num=100) 
            XX_scaled = XX.copy()
            pdep, confi = gam.partial_dependence(term=term_idx, width=0.95, X=XX_scaled)
            ax = axis[i] if len(numerical_terms) > 1 else axis
            ax.plot(XX[:, term.feature], pdep)
            ax.plot(XX[:, term.feature], confi, c='#3276b5', ls='--')
            ax.fill_between(XX[:, term.feature], confi[:, 0], confi[:, 1], color='#3276b5', alpha=0.2)
            ax.set_title(feature_name)
            ax.grid(linestyle='--', alpha=0.7, linewidth=0.5)

        if save_prefix:
            fig_numerical.savefig(f"{save_prefix}_numerical_term_partial_dependence.pdf", bbox_inches='tight')

    # plot categorical terms as bar plots
    if len(categorical_terms) > 0:

        fig_categorical, axis = plt.subplots(figsize=(10, 2), nrows=1, ncols=len(categorical_terms))
        for i, term_idx in enumerate(categorical_terms):

            term = terms[term_idx]
            feature_name = X.columns[term.feature]
            categories = sorted(X[feature_name].unique())

            XX = np.zeros((len(categories), len(X.columns)))
            XX[:, term.feature] = np.arange(len(categories))
            XX_scaled = XX.copy()
            pdep, confi = gam.partial_dependence(term=term_idx, width=0.95, X=XX_scaled)
            ax = axis[i] if len(categorical_terms) > 1 else axis
            # add confidence intervals
            ax.errorbar(XX[:, term.feature], pdep, yerr=np.abs(confi.T), fmt='o', color='#3276b5', capsize=5)
            ax.scatter(XX[:, term.feature], pdep, s=100, alpha=0.7, zorder=100)
            ax.set_title(feature_name)
            ax.set_xticks(np.arange(len(categories)), labels=categories)
            ax.set_xlim(-0.5, len(categories) - 0.5)
            ax.grid(axis='y', linestyle='--', alpha=0.7, linewidth=0.5)

        if save_prefix:
            fig_categorical.savefig(f"{save_prefix}_categorical_term_partial_dependence.pdf", bbox_inches='tight')

    # plot the tensor terms as 2D heatmaps
    # fig_tensor, axis = plt.subplots(figsize=(10, 4), nrows=1, ncols=len(tensor_terms))
    # create a subplots of two cols not equal ratio
    if len(tensor_terms) > 0:
        fig_tensor, axis = plt.subplots(figsize=(10, 4), nrows=1, ncols=len(tensor_terms), gridspec_kw={'width_ratios': [1, 2]})
        for i, term_idx in enumerate(tensor_terms):
            term = terms[term_idx]
            feature_names = [X.columns[feat] for feat in term.feature]
            feature_ranges = [(df[feat].min(), df[feat].max()) for feat in feature_names]

            XX = np.zeros((100 * 100, len(X.columns)))
            XX[:, term.feature] = np.array(np.meshgrid(*(np.linspace(*r, 100) for r in feature_ranges))).reshape(-1, 100*100).T
            XX_scaled = XX.copy()
            pdep, confi = gam.partial_dependence(term=term_idx, width=0.95, X=XX_scaled)
            
            ax = axis[i] if len(tensor_terms) > 1 else axis
            assert isinstance(ax, plt.Axes) 
            ax.set_title(' × '.join(feature_names))

            levels = 50

            if feature_names == ['long', 'lat']:
                centroid = np.array([c.coords[0] for c in geodata['geometry'].centroid])
                XX_centroid = np.zeros((len(centroid), len(X.columns)))
                XX_centroid[:, term.feature] = centroid
                XX_centroid_scaled = XX_centroid.copy()
                pdep_centroid, confi = gam.partial_dependence(term=term_idx, width=0.95, X=XX_centroid_scaled)

                vmin = (pdep_centroid.min() + pdep.min()) / 2
                vmax = (pdep_centroid.max() + pdep.max()) / 2

                geodata['color'] = pdep_centroid
                ax.set_axis_off()

            else:
                vmin = pdep.min()
                vmax = pdep.max() 

            contour = ax.contourf(XX[:, term.feature[0]].reshape(100, 100),
                                XX[:, term.feature[1]].reshape(100, 100),
                                pdep.reshape(100, 100), levels=levels, cmap='Blues', alpha=0.8,
                                vmin=vmin, vmax=vmax)
            
            if feature_names == ['long', 'lat']:
                geodata.plot(ax=ax, column='color', cmap='Blues', edgecolor='white', linewidth=0.5, alpha=0.8, vmin=vmin, vmax=vmax)

            plt.colorbar(contour, ax=ax, orientation='vertical')

            ax.set_xlim(feature_ranges[0])
            ax.set_ylim(feature_ranges[1])
            ax.set_xlabel(feature_names[0])
            ax.set_ylabel(feature_names[1])
            ax.grid(linestyle='--', alpha=0.7, linewidth=0.5)

        if save_prefix:
            fig_tensor.savefig(f"{save_prefix}_tensor_term_partial_dependence.pdf", bbox_inches='tight')

    return fig_numerical, fig_categorical, fig_tensor

plot_partial_dependence(models["GAM"][1].frequency_model, X_freq, save_prefix="frequency")
plot_partial_dependence(models["GAM"][1].severity_model, X_sev, save_prefix="severity")


In [ ]:
model_results = {
    "GAM": pd.read_pickle("gam_results.pkl"),
    "GLM (binned)": pd.read_csv("glm_results.csv"),
    "GLM (original)": pd.read_csv("glm_original_results.csv"),
    "GBM (binned)": pd.read_csv("gbm_results.csv"),
    "GBM (original)": pd.read_csv("gbm_original_results.csv"),
    "FFNN (binned)": pd.read_csv("ffnn_pytorch_results.csv"),
    "FFNN (original)": pd.read_csv("ffnn_original_results.csv"),
    "MTNN (binned)": pd.read_csv("multitask_results_final2.csv"),
    "MTNN (original)": pd.read_csv("mtl_original_results_final2.csv"),
    "MTMoE (binned)": pd.read_csv("moe_multitask_results_test2.csv"),
    "MTMoE (original)": pd.read_csv("moe_original_results_test2.csv")
}

In [ ]:
observed_loss = df.groupby('fold').agg({"amount": "sum"})['amount']

In [ ]:
average_results_df = pd.DataFrame(index=list(model_results.keys()), columns=["predicted_loss", "freq_deviance", "sev_deviance", "price_ratios"])
freq_deviance_df = pd.DataFrame(index=list(model_results.keys()), columns=[f"Fold {i}" for i in range(1, 7)])
sev_deviance_df = pd.DataFrame(index=list(model_results.keys()), columns=[f"Fold {i}" for i in range(1, 7)])
price_ratios_df = pd.DataFrame(index=list(model_results.keys()), columns=[f"Fold {i}" for i in range(1, 7)])

model_names = []
for model_name, model_df in model_results.items():
    model_names.append(model_name)
    model_df = model_df.set_index("fold")
    model_df["predicted_loss"] = observed_loss * model_df["price_ratios"]
    average_results_df.loc[model_name] =  model_df.agg({
        "predicted_loss": "sum",
        "freq_deviance": "mean",
        "sev_deviance": "mean",
        "price_ratios": "mean",
    })
    freq_deviance_df.loc[model_name, :] = model_df["freq_deviance"].values
    sev_deviance_df.loc[model_name, :] = model_df["sev_deviance"].values
    price_ratios_df.loc[model_name, :] = model_df["price_ratios"].values

freq_deviance_df["Mean"] = freq_deviance_df.mean(axis=1)
sev_deviance_df["Mean"] = sev_deviance_df.mean(axis=1)
price_ratios_df["Mean"] = price_ratios_df.mean(axis=1)
average_results_df['predicted_loss'] = average_results_df['predicted_loss'].astype(int)
display(average_results_df)
display(freq_deviance_df)
display(sev_deviance_df)
display(price_ratios_df)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 5))
plt.subplots_adjust(wspace=0.1)  # Add right margin

for i, (ax, key, title) in enumerate(zip(axes, ["price_ratios", "freq_deviance", "sev_deviance"], ["Price Ratios Comparison", "Frequency Deviance Comparison", "Severity Deviance Comparison"])):
    compare_df = pd.DataFrame({
        "GAM": gam_results_df[key],
        "GLM (binned)": glm_binned_results_df[key],
        "GLM (original)": glm_original_results_df[key],
        "GBM (binned)": gbm_binned_results_df[key],
        "GBM (original)": gbm_original_results_df[key],
        "FFNN (binned)": ffnn_binned_results_df[key],
        "FFNN (original)": ffnn_original_results_df[key],
        "MTL (binned)": mtnn_binned_results_df[key],
        "MTL (original)": mtnn_original_results_df[key],
        "MTL-MOE (binned)": mtmoe_binned_results_df[key],
        "MTL-MOE (original)": mtmoe_original_results_df[key],
    })

    melted_df = compare_df.melt(var_name="Model", value_name=key)
    sns.boxplot(data=compare_df, orient="h", palette="Set2", ax=ax)
    sns.stripplot(data=melted_df, x=key, y="Model", hue="Model", palette="Set2", size=8, alpha=1, linewidth=0.5, jitter=False, ax=ax)

    n_folds, n_methods = compare_df.shape
    for fold in range(n_folds):
        ax.plot(compare_df.iloc[fold, :], np.arange(n_methods), marker="o", linestyle="--", alpha=0.2, linewidth=1, color="black", markersize=4)

    ax.set_title(title, fontsize=14)
    ax.set_xlabel("")
    if i > 0:
        ax.set_ylabel("")
        ax.set_yticks([])

    ax.tick_params(axis="y", labelsize=9)
    ax.grid(axis="x", alpha=0.3)

axes[0].axvline(x=1.0, color="black", linestyle="--", alpha=0.8, linewidth=2, label="Perfect Price Ratio (1.0)", zorder=-10)
fig.savefig("price_ratio_and_deviance_comparison.pdf", bbox_inches="tight", dpi=300, pad_inches=0.3)

In [ ]:
def plot_expected_vs_predicted(y_pred, y_true, bins=100, delta=None, ax=None, y_min=None, y_max=None, line_kwargs={}):
    if ax is None:
        ax = plt.gca()
    if y_min is None:
        y_min = y_pred.min()
    if y_max is None:
        y_max = y_pred.max()
    if delta is None:
        delta = (y_max - y_min) / 10

    bins = 100
    S = np.linspace(y_min, y_max, bins)
    E = np.array([y_true[(y_pred >= s - delta) & (y_pred <= s + delta)].mean() for s in S])

    ax.plot(S, E, **line_kwargs)
    ax.plot([y_min, y_max], [y_min, y_max], color="black", linestyle="--", linewidth=2)


def plot_histogram(y_pred, bins=100, ax=None, y_min=None, y_max=None, hist_kwargs={}, kde_kwargs={}):
    if ax is None:
        ax = plt.gca()
    if y_min is None:
        y_min = y_pred.min()
    if y_max is None:
        y_max = y_pred.max()

    ax.hist(y_pred, bins=bins, range=(y_min, y_max), **hist_kwargs, density=True)
    ax.set_xlim(y_min, y_max)
    sns.kdeplot(y_pred, ax=ax, **kde_kwargs)


In [ ]:
cmap = sns.color_palette("Dark2", n_colors=len(models))
cmap

In [ ]:
test_predictions = {}
for fold in range(1, 7):
    test_mask = df["fold"] == fold

    for i, (name, m) in enumerate(models.items()):
        freq_pred, sev_pred = m[fold].predict(X_freq[test_mask], X_sev[test_mask])
        price_ratio = float((freq_pred * sev_pred * df["expo"][test_mask]).sum() / df["amount"][test_mask].sum())
        sev_pred = sev_pred[sev_mask[test_mask]]
        freq_true = df["nclaims"][test_mask].to_numpy() / df["expo"][test_mask].to_numpy()
        sev_true = df["average"][test_mask & sev_mask].to_numpy()
        test_predictions[(fold, name)] = (freq_pred, sev_pred, freq_true, sev_true)

In [ ]:
fig1, axes1 = plt.subplots(figsize=(15, 5), ncols=6, nrows=2)
fig2, axes2 = plt.subplots(figsize=(15, 5), ncols=6, nrows=2)

fig1.subplots_adjust(hspace=0.2)
fig2.subplots_adjust(wspace=0.4, hspace=0.4)


for i, (name, m) in enumerate(models.items()):
    freq_preds = []
    sev_preds = []
    for fold in range(1, 7):
        freq_pred, sev_pred, freq_true, sev_true = test_predictions[(fold, name)]
        plot_expected_vs_predicted(freq_pred, freq_true, bins=100, ax=axes1[0, i], y_min=0.05, y_max=0.2, line_kwargs=dict(color=cmap[i], linewidth=2, alpha=0.5))
        plot_expected_vs_predicted(sev_pred, sev_true, bins=100, ax=axes1[1, i], y_min=1000, y_max=1400, delta=100, line_kwargs=dict(color=cmap[i], linewidth=2, alpha=0.5))
        freq_preds.append(freq_pred)
        sev_preds.append(sev_pred)

    plot_histogram(np.hstack(freq_preds), bins=100, ax=axes2[0, i], hist_kwargs=dict(color=cmap[i], alpha=0.5), kde_kwargs=dict(color=cmap[i], alpha=0.8))
    plot_histogram(np.hstack(sev_preds), bins=100, ax=axes2[1, i], hist_kwargs=dict(color=cmap[i], alpha=0.5), kde_kwargs=dict(color=cmap[i], alpha=0.8))


freq_range = [0.05, 0.2]
sev_range = [1000, 1400]
freq_ticks = np.arange(0.05, 0.21, 0.05)
sev_ticks = np.arange(1050, 1500, 150)
freq_density_max = 18
sev_density_max = 0.007

for i, name in enumerate(models):
    axes1[0, i].set_xlim(freq_range)
    axes1[0, i].set_ylim(freq_range)
    axes1[0, i].set_xticks(freq_ticks)
    axes1[0, i].set_title(name, fontsize=12)
    axes1[0, i].grid(axis="both", alpha=0.3)

    axes1[1, i].set_xticks(sev_ticks)
    axes1[1, i].set_yticks(sev_ticks)
    axes1[1, i].set_xlim(sev_range)
    axes1[1, i].set_ylim(sev_range)
    axes1[1, i].grid(axis="both", alpha=0.3)

    axes2[0, i].set_title(name, fontsize=12)
    axes2[0, i].set_xlabel("Frequency", fontsize=10)
    axes2[1, i].set_xlabel("Severity", fontsize=10)
    axes2[0, i].set_xlim(freq_range)
    axes2[0, i].set_xticks(freq_ticks)
    axes2[1, i].set_xlim(sev_range)
    axes2[1, i].set_xticks(sev_ticks)
    axes2[0, i].set_ylim(0, freq_density_max)
    axes2[1, i].set_ylim(0, sev_density_max)

    if i > 0:
        axes1[0, i].set_yticklabels([])
        axes1[1, i].set_yticklabels([])
        axes1[0, i].set_ylabel("")
        axes1[1, i].set_ylabel("")
        axes2[0, i].set_ylabel("")
        axes2[1, i].set_ylabel("")

axes1[0, 0].set_ylabel("Frequency")
axes1[1, 0].set_ylabel("Severity")
axes2[0, 0].set_ylabel("Density")
axes2[1, 0].set_ylabel("Density")

fig1.text(x=0.06, y=0.35, s="Average Observation", fontsize=12, rotation=90)
fig1.text(x=0.45, y=0, s="Predicted Expectation", fontsize=12, rotation=0)
fig1.savefig("expected_vs_predicted_freq_sev.pdf", bbox_inches="tight")
fig2.savefig("histogram_freq_sev.pdf", bbox_inches="tight")

$$
\mathbb{E}[Y \mid f(X) \in [s- \delta , s + \delta]]
$$

$$
\{(s, ~ \mathbb{E}[y_\text{true} \mid y_\text{pred} \in [s - \delta , s + \delta]) ~ |~ s \in S \}
$$

In [ ]:
def set_col(df, col, value):
    if col in df.columns:
        new_df = df.copy()
        new_df.loc[:, col] = value
        return new_df
    return df


def random_col(df, col):
    if col in df.columns:
        return set_col(df, col, np.random.permutation(df[col].values))
    return df


In [ ]:
features = ["coverage", "fuel", "ageph", "power", "bm", "long", "lat"]

In [ ]:
np.random.seed(42)

freq_VIP_df = pd.DataFrame(index=features, columns=list(models.keys()))
sev_VIP_df = pd.DataFrame(index=features, columns=list(models.keys()))
freq_VIP_df.loc[:, :] = 0
sev_VIP_df.loc[:, :] = 0

for fold in trange(1, 7):
    for name, m in models.items():
        for feat in features:
            X_freq_rand = random_col(X_freq, feat)
            X_sev_rand = random_col(X_sev, feat)
            freq_pred, sev_pred = m[fold].predict(X_freq, X_sev)
            freq_pred_rand, sev_pred_rand = m[fold].predict(X_freq_rand, X_sev_rand)
            freq_VIP_df.loc[feat, name] += np.sum(np.abs(freq_pred_rand - freq_pred))
            sev_VIP_df.loc[feat, name] += np.sum(np.abs(sev_pred_rand - sev_pred))

freq_VIP_df /= freq_VIP_df.sum(axis=0)
sev_VIP_df /= sev_VIP_df.sum(axis=0)

freq_VIP_df.to_csv("freq_VIP.csv")
sev_VIP_df.to_csv("sev_VIP.csv")


In [ ]:
freq_VIP_df = pd.read_csv("freq_VIP.csv", index_col=0)
sev_VIP_df = pd.read_csv("sev_VIP.csv", index_col=0)

fig, axes = plt.subplots(figsize=(10, 6), nrows=1, ncols=2)
fig.subplots_adjust(wspace=0.1)

def plot_vip(df, ax, title):
    df = df.reset_index().melt(id_vars='index', var_name='Model', value_name='VIP')
    sns.barplot(data=df, y='index', x='VIP', hue='Model', palette='Dark2', alpha=0.6, ax=ax, orient='h')
    ax.set_title(title)
    ax.set_ylabel("Features")
    ax.set_xlabel("VIP")
    ax.set_yticks(range(len(features)), features)
    ax.get_legend().remove()  # Remove individual legends

plot_vip(freq_VIP_df, axes[0], "Frequency Variable Importance Plot (VIP)")
plot_vip(sev_VIP_df, axes[1], "Severity Variable Importance Plot (VIP)")

axes[1].set_yticklabels([])
axes[1].set_ylabel("")

axes[0].grid(axis='both', alpha=0.4)
axes[1].grid(axis='both', alpha=0.4)

# Create a common legend
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=len(labels))
fig.savefig("vip_scores.pdf", bbox_inches="tight")

In [ ]:
freq_significant_features = freq_VIP_df.sum(axis=1).sort_values(ascending=False)[:2].index.tolist()
sev_significant_features = sev_VIP_df.sum(axis=1).sort_values(ascending=False)[:2].index.tolist()
print("Most significant frequency features:", freq_significant_features)
print("Most significant severity features:", sev_significant_features)

# significant_features = ["bm", "ageph"]  # Use the most significant features for PD calculation
significant_features = ["bm", "ageph"]  # Use the most significant features for PD calculation


In [ ]:
freq_PD: dict[str, dict[str, dict[float, float]]] = {}
sev_PD: dict[str, dict[str, dict[float, float]]] = {}

pbar = tqdm(total=len(models) * len(significant_features), desc="Calculating PD scores", unit="feat")

for feat in significant_features:
    freq_PD[feat] = {}
    sev_PD[feat] = {}

    # if categorical
    if X_freq[feat].dtype == "categorical":
        possible_values = X_freq[feat].cat.categories
    # if integer 
    elif X_freq[feat].dtype in ("int64", "int32"):
        possible_values = np.unique(X_freq[feat])
    elif X_freq[feat].dtype == "float64":
        possible_values = np.linspace(X_freq[feat].min(), X_freq[feat].max(), 10)
    else:
        raise ValueError(f"Unsupported feature type for {feat}: {X_freq[feat].dtype}")
    
    for name, m in models.items():
        freq_PD[feat][name] = {}
        sev_PD[feat][name] = {}

        pbar.set_description(f"Calculating PD scores for {name}", refresh=True)

        for value in possible_values:
            freq_PD[feat][name][value] = 0.0
            sev_PD[feat][name][value] = 0.0
            
            for fold in range(1, 7):
                X_freq_rand = set_col(X_freq, feat, value)
                X_sev_rand = set_col(X_sev, feat, value)
                freq_pred, sev_pred = m[fold].predict(X_freq_rand, X_sev_rand)
                freq_PD[feat][name][value] += freq_pred.mean()
                sev_PD[feat][name][value] += sev_pred.mean()
            freq_PD[feat][name][value] /= 6
            sev_PD[feat][name][value] /= 6

        pbar.update(1)


pickle.dump(freq_PD, open("freq_PD.pkl", "wb"))
pickle.dump(sev_PD, open("sev_PD.pkl", "wb"))

In [ ]:
freq_PD = pickle.load(open("freq_PD.pkl", "rb"))
sev_PD = pickle.load(open("sev_PD.pkl", "rb"))

fig, axes = plt.subplots(figsize=(10, 6), nrows=2, ncols=len(significant_features))
fig.subplots_adjust(wspace=0.2, hspace=0.1)
for i, feat in enumerate(significant_features):
    is_categorical = X_freq[feat].dtype == "category"
    for j, name in enumerate(models):
        if is_categorical:
            raise NotImplementedError("Categorical features not supported in this plot.")
        else:
            axes[0, i].plot(freq_PD[feat][name].keys(), freq_PD[feat][name].values(), label=name, color=cmap[j])
            axes[1, i].plot(sev_PD[feat][name].keys(), sev_PD[feat][name].values(), label=name, color=cmap[j])
    
    axes[0, i].grid(axis='both', alpha=0.4)
    axes[1, i].grid(axis='both', alpha=0.4)
    axes[0, i].set_xlim(X_freq[feat].min(), X_freq[feat].max())
    axes[1, i].set_xlim(X_freq[feat].min(), X_freq[feat].max())
    axes[-1, i].set_xlabel(feat)
    
axes[0, 0].set_xticklabels([])
axes[0, 1].set_xticklabels([])

axes[0, 0].set_ylabel("Frequency")
axes[1, 0].set_ylabel("Severity")


# Create a common legend
handles, labels = axes[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=len(labels))
fig.savefig("partial_dependence_plots.pdf", bbox_inches="tight")